In [1]:
import pandas as pd
import json
import numpy as np
import langid
import re
import emot
from nltk.tokenize import casual

In [184]:
with open('testing.json') as json_data:
    data = json.load(json_data)

In [185]:
posts_df = pd.DataFrame()

def get_columns(data):
    
    for subreddit,post_dict in data.items():
        for title,comm_list in post_dict.items():
            titles.append(title)
#             for comment in comm_list:
#                 print(type(comment))
#                 print(len(comment))
            comments.append(comm_list)
    return titles, comments
    
titles = []
comments = []
titles, comments = get_columns(data)
posts_df['titles']=titles
posts_df['comments']=comments

# these are text cleaning utils, 
    - make a utils pipeline later

# remove posts that have no comments

In [186]:
posts_df = posts_df[posts_df.astype(str)['comments'] != '[]']

In [187]:
posts_df.head()

,titles,comments
0,I Was With Bernie Till the End; Now We All Mus...,"[\nAs a reminder, this subreddit [is for civil..."
1,"[Discussion] State of the Union, 9 pm EST",[[Post-State of the Union discussion thread](h...
2,"My Writing in The Huffington Post, Salon, and ...","[The AMA has ended, and we've locked the comme..."
3,Vote up if you think Bush should be removed fr...,"[I hate you., We don't do it this way., Obliga..."
4,"Hi Reddit, I am Maria Teresa Kumar, Emmy-nomin...",[Are you surprised in the record low public tr...


## Convert 2nd row from list to string

In [188]:
posts_df.reset_index(inplace=True)

In [189]:
comments_str = []
for i in range(len(posts_df)):
     comments_str.append([i,str(posts_df.comments[i][0])])

In [190]:
comments = pd.DataFrame(np.array(comments_str))

In [191]:
posts_df = posts_df.join(comments)

In [192]:
posts_df.head()

,index,titles,comments,0,1
0,0,I Was With Bernie Till the End; Now We All Mus...,"[\nAs a reminder, this subreddit [is for civil...",0,"\nAs a reminder, this subreddit [is for civil ..."
1,1,"[Discussion] State of the Union, 9 pm EST",[[Post-State of the Union discussion thread](h...,1,[Post-State of the Union discussion thread](ht...
2,2,"My Writing in The Huffington Post, Salon, and ...","[The AMA has ended, and we've locked the comme...",2,"The AMA has ended, and we've locked the commen..."
3,3,Vote up if you think Bush should be removed fr...,"[I hate you., We don't do it this way., Obliga...",3,I hate you.
4,4,"Hi Reddit, I am Maria Teresa Kumar, Emmy-nomin...",[Are you surprised in the record low public tr...,4,Are you surprised in the record low public tru...


In [193]:
posts_df = posts_df.drop(['index', 'comments',0], axis=1)
posts_df = posts_df.rename(columns = {1:'comments'})
posts_df

,titles,comments
0,I Was With Bernie Till the End; Now We All Mus...,"\nAs a reminder, this subreddit [is for civil ..."
1,"[Discussion] State of the Union, 9 pm EST",[Post-State of the Union discussion thread](ht...
2,"My Writing in The Huffington Post, Salon, and ...","The AMA has ended, and we've locked the commen..."
3,Vote up if you think Bush should be removed fr...,I hate you.
4,"Hi Reddit, I am Maria Teresa Kumar, Emmy-nomin...",Are you surprised in the record low public tru...
...,...,...
397,A friendly reminder to vote democratic tomorrow,Im sorry but PLEASE DO NOT JUST VOTE PARTY LIN...
398,Community project: Open letter to r/atheism o...,"This 'open letter' isn't, I feel, quite repres..."
399,Justin Bieber: Being gay is a decision and abo...,1. This headline is shamelessly pushing the lo...
400,First time I made out with a bi-girl. [Rage Co...,"""First time I ever made out with an Estonian, ..."


# Processing emoji's

In [81]:
emoji_df = pd.read_csv('~/code/ctt10/Uncancelled/Emoji_Unicode_v13.txt', sep='?', engine='python', header=None)
emoji_df.head()

,0,1,2
0,1F600,fully-qualified,😀 E1.0 grinning face
1,1F603,fully-qualified,😃 E0.6 grinning face with big eyes
2,1F604,fully-qualified,😄 E0.6 grinning face with smiling eyes
3,1F601,fully-qualified,😁 E0.6 beaming face with smiling eyes
4,1F606,fully-qualified,😆 E0.6 grinning squinting face


In [82]:
emoji_df['emoji'], emoji_df['description'] = emoji_df[2].str.split('E', 1).str
emoji_df[0] = emoji_df[0].apply(lambda x: x.lower())
emoji_df.head()

/home/rtc/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


,0,1,2,emoji,description
0,1f600,fully-qualified,😀 E1.0 grinning face,😀,1.0 grinning face
1,1f603,fully-qualified,😃 E0.6 grinning face with big eyes,😃,0.6 grinning face with big eyes
2,1f604,fully-qualified,😄 E0.6 grinning face with smiling eyes,😄,0.6 grinning face with smiling eyes
3,1f601,fully-qualified,😁 E0.6 beaming face with smiling eyes,😁,0.6 beaming face with smiling eyes
4,1f606,fully-qualified,😆 E0.6 grinning squinting face,😆,0.6 grinning squinting face


In [83]:
def clean_num(text):
    #text = ''.join(char for char in text if not char.isdigit())
    text = text.str.replace('\d+', '')
    text = text.str.replace('.', '')
    return text
emoji_df['description']= clean_num(emoji_df['description'])

In [84]:
def make_col(df):
    df=df.drop([1,2, 'emoji'], axis =1)
    df=df.rename(columns = {0:'unicode'})
    return df
emoji_df= make_col(emoji_df)

In [85]:
emoji_dict = emoji_df.set_index('unicode').to_dict()

In [86]:
emoji_dict = emoji_dict['description']

In [146]:
emoji_dict

{'1f600                                      ': ' grinning face',
 '1f603                                      ': ' grinning face with big eyes',
 '1f604                                      ': ' grinning face with smiling eyes',
 '1f601                                      ': ' beaming face with smiling eyes',
 '1f606                                      ': ' grinning squinting face',
 '1f605                                      ': ' grinning face with sweat',
 '1f923                                      ': ' rolling on the floor laughing',
 '1f602                                      ': ' face with tears of joy',
 '1f642                                      ': ' slightly smiling face',
 '1f643                                      ': ' upside-down face',
 '1f609                                      ': ' winking face',
 '1f60a                                      ': ' smiling face with smiling eyes',
 '1f607                                      ': ' smiling face with halo',
 '1f970    

# convert emoji/emote to text

In [123]:
a = "\u1F600 Until then, there's no reasonable reason to believe so.;, \u1F603 ;\u00e6z \u00f0\u0259 n\u028cmb\u025a \u028cv g\u0251dz \u026ankrisez, \u00f0\u0259 pr\u0251b\u028cb\u026al\u026ati \u028cv t\u0283uzi\u014b \u00f0\u0259 k\u0254rekt w\u028cn \u028cprout\u0283\u026az zirou."

In [194]:
def remove_u(word):
    word_u = (word.encode('unicode-escape')).decode("utf-8", "strict")
    if r'\u' in word_u: 
        # print(True)
        return word_u.replace('\\u', '')
    return word

posts_df['comments'] = posts_df.comments.apply(lambda x: remove_u(x))
posts_df

,titles,comments
0,I Was With Bernie Till the End; Now We All Mus...,"\nAs a reminder, this subreddit [is for civil ..."
1,"[Discussion] State of the Union, 9 pm EST",[Post-State of the Union discussion thread](ht...
2,"My Writing in The Huffington Post, Salon, and ...","The AMA has ended, and we've locked the commen..."
3,Vote up if you think Bush should be removed fr...,I hate you.
4,"Hi Reddit, I am Maria Teresa Kumar, Emmy-nomin...",Are you surprised in the record low public tru...
...,...,...
397,A friendly reminder to vote democratic tomorrow,Im sorry but PLEASE DO NOT JUST VOTE PARTY LIN...
398,Community project: Open letter to r/atheism o...,"This 'open letter' isn't, I feel, quite repres..."
399,Justin Bieber: Being gay is a decision and abo...,1. This headline is shamelessly pushing the lo...
400,First time I made out with a bi-girl. [Rage Co...,"""First time I ever made out with an Estonian, ..."


In [195]:
def emoji_to_text(text):
    for k, v in emoji_dict.items():
        k = k.rstrip()
        if k in text:
            text = text.replace(k,v)
    return text

posts_df['comments'] = posts_df.comments.apply(lambda x: emoji_to_text(x))

In [197]:
posts_df.head()

,titles,comments
0,I Was With Bernie Till the End; Now We All Mus...,"\nAs a reminder, this subreddit [is for civil ..."
1,"[Discussion] State of the Union, 9 pm EST",[Post-State of the Union discussion thread](ht...
2,"My Writing in The Huffington Post, Salon, and ...","The AMA has ended, and we've locked the commen..."
3,Vote up if you think Bush should be removed fr...,I hate you.
4,"Hi Reddit, I am Maria Teresa Kumar, Emmy-nomin...",Are you surprised in the record low public tru...


# Export

In [198]:
posts_df.to_csv('~/code/ctt10/Uncancelled/reddit_posts.csv')

# Removing Numbers

In [19]:
posts_df.astype(str)

,titles,comments
0,Market Failure Visualized,['This post has been marked nsfw for apparent ...
1,Are AirBnb and Uber/Lyft a symptom?,[]
2,"Инресен въпрос включващ един арменец, един бъл...",['Boo.']
3,The Darkside of 13 of History's Favourite People,['Lol at newton compared to the rest. Also FDR...
4,r/AskHistorians: George Floyd was murdered by ...,[]
...,...,...
234,"Right wing grifter guest Pim Tool, doubling do...","[""1. IDK if this is really Joe Rogan material\..."
235,'We will print more money and give it to the u...,"['So I looked this up, and this unknown websit..."
236,"Trump closes out campaign with inspiring, poig...","[""I'll put it this way. \n\nIf I found out Tr..."
237,New footage shows it was the Native American g...,"[""woudln't be the first time a social media fl..."


In [24]:
def clean_num(text):
    #text = ''.join(char for char in text if not char.isdigit())
    text = text.str.replace('\d+', '')
    return text
posts_df['comments']= clean_num(posts_df.comments)
posts_df['titles']= clean_num(posts_df.titles)

# Remove non-English

In [25]:
def remove_non_en(text):
    '''takes a pandas series'''
    return text.apply(lambda x: x if langid.classify(x)=='en' else None)
posts_df['titles'] = remove_non_en(posts_df.titles)
posts_df['comments'] = remove_non_en(posts_df.comments)


TypeError: 'float' object is not iterable

## Lower Case

In [433]:
posts_df.comments[0]

['This post has been marked nsfw for apparent reasons.  It is recommended that next time you create an overbearing post with implications similar to that of a visual laxative, you will have the common decency to mark it nsfw yourself.',
 'Ooh']

In [444]:
def df_lower(series):
    for comm_list in series:
        #3print(comm_list)
        for char in comm_list:
            return char.lower()
            #for letter in comm_list:
                #print(letter)
                #letter.lower()
some_text = df_lower(posts_df.comments)
#print(some_text)

In [445]:
some_text

'this post has been marked nsfw for apparent reasons.  it is recommended that next time you create an overbearing post with implications similar to that of a visual laxative, you will have the common decency to mark it nsfw yourself.'

In [401]:
len(posts_df)

TypeError: object of type 'NoneType' has no len()

In [234]:
comm_list

['[deleted]',
 'Trump claimed he saw thousands of people **in NJ** celebrating. I suspect he saw the footage below and thought it was NJ, or misremembered where it was from. \n\nWhile I did not see anyone *in the US* celebrating on 9/11, I did see [video of Palestinians celebrating](https://www.youtube.com/watch?v=G-p1LEBAujE). (video from FOX, CNN, and MSNBC)  And no, the video was [not footage from years before](http://www.snopes.com/rumors/cnn.asp).\n\nThe other things on the list, well, Trump is a liar. That much is not news.',
 "This isn't news.",
 'How is this "in the news"?',
 '>Lying liar lying out of his lying lair.\n\nFTFY',
 "Man it's so easy to post things like this without any proof",
 'Can I get a source on that?',
 ">there was no incident in Sweden that night\n\nBad luck for picking the ONE day all the agitators decided to get a good night's sleep.",
 '[deleted]',
 'You must sleep well if you believe all of that. ',
 '[deleted]',
 '> Trump claimed he saw thousands of peo

In [233]:
posts_df.comments[0]

['This post has been marked nsfw for apparent reasons.  It is recommended that next time you create an overbearing post with implications similar to that of a visual laxative, you will have the common decency to mark it nsfw yourself.',
 'Ooh']

## Remove Stop words

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
stop_words.remove('are')
stop_words.remove('again')
stop_words.remove('once')
stop_words.remove('no')
stop_words.remove('only')
stop_words.remove('most')
stop_words.remove('very')
stop_words.remove('will')
stop_words.remove('don')
stop_words.remove('should')

def remove_stop(text):
    word_tokens = word_tokenize(text)
    text = [w for w in word_tokens if not w in stop_words]
    return ' '.join(text)
posts_df['clean_comments']= posts_df.clean_comments.apply(remove_stop)

## remove non-english

In [209]:
def Remove_non_en(text):
    '''takes a pandas series'''
    return text.apply(lambda x: x if langid.classify(x)[0]=='en' else None)
posts_df['clean_comments'] = posts_df['clean_comments'].apply(remove_non_en())

NameError: name 'train_data' is not defined

## remove punctuation
    -- deal with punctuation like apostrophes
        - change contractions by replacing words

In [ ]:
def clean(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
        text.lower()
    return text

posts_df['clean_text']= posts_df.clean_text.apply(clean)

## Replace contractions

In [ ]:
def clean_string(x):
    x = x.replace('i m', 'i am') #(old, new, count)
    x = x.replace('he s', 'he is')
    x = x.replace('she s', 'she is')
    x = x.replace('you re', 'you are')
    x = x.replace('we re', 'we are')
    x = x.replace('they re', 'they are')
    x = x.replace('can t', 'can not')
    x = x.replace('won t', 'will not')
    x = x.replace('i ll', 'i will')
    x = x.replace('you ve', 'you have')
    x = x.replace('we ve', 'we have')
    x = x.replace('they ll', 'they will') 
    x = x.replace('there s', 'there is') 
    x = x.replace('that s', 'that is') 
    x = x.replace('aren t', 'are not')
    x = x.replace('don t', 'do not') 
    x = x.replace('shouldn t', 'should not')
    x = x.replace('wasn t', 'was not') 
    x = x.replace('wouldn t', 'would not') 
    return x
posts_df['clean_comments'] = posts_df.clean_text.apply(lambda x: clean_string(x))

## memoving most common words

In [ ]:
# Checking the first 10 most frequent words
from collections import Counter
def remove_common(text):
    cnt = Counter()
    for text in df["clean_comment"]:
        for word in text.split():
            cnt[word] += 1
    return cnt
        
cnt.most_common(10)

## lemmantize words

In [66]:
def lemm(text):
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    return ' '.join(text)
posts_df['clean_text']= posts_df.clean_text.copy().apply(lemm)